In [1]:
import cv2
import numpy as np
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import skimage.io as io

In [2]:
def fit_elliptical_boundary_model(features):
    # Fit a Gaussian Mixture Model (GMM) to the features
    gmm = GaussianMixture(n_components=1, covariance_type='full')
    print(features.size)
    gmm.fit(features) 
    # Get the parameters of the fitted Gaussian component
    mean = gmm.means_[0]
    cov_matrix = gmm.covariances_[0]

    # Calculate the eigenvalues and eigenvectors of the covariance matrix
    eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)

    # Sort the eigenvalues and eigenvectors in descending order
    indices = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[indices]
    eigenvectors = eigenvectors[:, indices]

    # Calculate the angle of rotation for the ellipse
    angle = np.degrees(np.arctan2(*eigenvectors[:, 0][::-1]))

    # Create an ellipse using matplotlib
    ellipse = Ellipse(mean, 2 * np.sqrt(5.991 * eigenvalues[0]),
                      2 * np.sqrt(5.991 * eigenvalues[1]), angle=angle,
                      fill=False, color='red', linewidth=2)

    return ellipse


In [3]:
def visualize_elliptical_boundary_model(image, ellipse):
    # Display the image with the fitted ellipse
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.gca().add_patch(ellipse)
    plt.axis('off')
    plt.show()

In [4]:
def main():
    # Load an example image
    image_path = "face.jpeg"
    image = cv2.imread(image_path)

    # Convert the image to YCrCb color space
    ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)

    # Extract the Cr and Cb channels as features
    features = ycrcb[:, :, 1:3].reshape((-1, 2))

    # Fit an elliptical boundary model to the features
    ellipse = fit_elliptical_boundary_model(features)

    # Visualize the elliptical boundary model
    visualize_elliptical_boundary_model(image, ellipse)

if __name__ == "__main__":
    main()

error: OpenCV(4.7.0) D:\bld\libopencv_1675730129252\work\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
